In [ ]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
from bs4 import BeautifulSoup
import time
import re
import string
import pandas as pd
#Match->Match URRL
driver = webdriver.Firefox()
url='http://premier.cricketvictoria.com.au/common/pages/public/rv/match.aspx?redirect=1&'
driver.get(url)
def find_teams(driver):
 #Find the Teams played for the tournament
        Teams = driver.find_element_by_xpath('//*[@id="ctl00_MainPlaceHolder2_RVPageTemplateControl_rvsb_selectorBar_inner"]/div/div[2]/table/tbody/tr/td[2]')

        dt=[]
        #Assumig max of 10 teams
        for i in range(1,15):
            if Teams.text.split('\n',i):     
                x=i-1
                try:
                      var=Teams.text.split('\n',i)[x]
                      dt.append(var)
                except:
                     x=1
        return dt
def find_grade(driver):
         #Select the Grade
    data=[]
    GG=driver.find_element_by_xpath('//*[@id="ctl00_MainPlaceHolder2_RVPageTemplateControl_rvsb_selectorBar_inner"]/div/div[1]/div[3]')
    
    for i in range(1,10): #loop for Grade
        if GG.text.split('\n',i):     
            x=i-1
            try:
                  var=GG.text.split('\n',i)[x]
                  data.append(var)
                  #print(data) 
            except:
                   x=1
    
    data.remove('Grade')
    return data
def find_round(driver):
    #select the Round
    Round=driver.find_element_by_xpath('//*[@id="ctl00_MainPlaceHolder2_RVPageTemplateControl_rvsb_selectorBar_inner"]/div/div[1]/div[2]')
    #print(Round.text)
    rd=[]
     #Assumig max of 20 rounds
    for i in range(1,20):
            if Round.text.split('\n',i):     
                x=i-1
                try:
                      var=Round.text.split('\n',i)[x]
                      rd.append(var)
                      #print(rd)
                except:
                         x=1
    rd.remove('Round')
    return rd

    

def batting_data(driver):
    
    html=driver.page_source
    soup=BeautifulSoup(html,'html.parser')
    pt=2

    #table = soup.find_elements_by_class_name('table', attrs={'class':'innings-table'})

    table=[T for T in soup.find_all(class_='innings-table')]
    Bowling_table=[T for T in soup.find_all(class_='bowling-table')]
    
    for table_index in range(0,len(table)):
            table_body=table[table_index].find('tbody')  
            Bowling_table_body=Bowling_table[table_index].find('tbody')
            data = []
            rows = table_body.find_all('tr')
            for row in rows:
                cols = row.find_all('td')
                cols = [ele.text.strip() for ele in cols]
                data.append([ele for ele in cols  ]) # Get rid of empty values
            #time.sleep(0.5)

            df=pd.DataFrame(data)
            #print(df) 
            #find date
            date=driver.find_element_by_xpath('/html/body/form/div[3]/div[1]/div[1]/div[2]/div[1]/div[2]/div/div/div/div/div[2]/table/tbody/tr[2]/td')
            dat=date.text
            dat=dat.split('(',1)[0]
            dat=dat.split('-',1)[1]
            #print(dat)
            List.append(dat)
            #match id
            Mid=driver.find_element_by_xpath('/html/body/form/div[3]/div[1]/div[1]/div[2]/div[1]/div[2]/div/div/div/div/div[2]/div[6]/div[1]/span')
            Mid=Mid.text.split(':',1)[1]
            #print(Mid)
        #print table index innings
            if table_index<=1:
                x=1
            elif table_index>1:
                x=2
      
            if table_index==0:
                #venue
                venue=driver.find_element_by_xpath('/html/body/form/div[3]/div[1]/div[1]/div[2]/div[1]/div[2]/div/div/div/div/div[2]/div[6]/div[1]/a')
                List.append(venue.text)
                #format of the game 
                form=driver.find_element_by_xpath('/html/body/form/div[3]/div[1]/div[1]/div[2]/div[1]/div[2]/div/div/div/div/div[2]/table/tbody/tr[2]/td')
                count=0
                for i in form.text:
                    if i== '(':
                        count+=1
                if count==1:
                    List.append('One day game')
                else:
                    List.append('Two day game')
                    
                    
                
                # who batted first
                Batted=driver.find_element_by_xpath('//*[@id="ctl00_MainPlaceHolder2_RVPageTemplateControl_panelMain"]/div[7]/div[1]/span[2]')
                sp=Batted.text.split(':',1)[1]

                sp=sp.split(" ",1)[1]
                #print("Batted first:",sp)
                #who won the match and batted first or second
                Defeat=driver.find_element_by_xpath('//*[@id="ctl00_MainPlaceHolder2_RVPageTemplateControl_panelMain"]/table/tbody/tr[3]/td[2]')
                if(Defeat.text=='def'):
                    Winner=driver.find_element_by_xpath('//*[@id="ctl00_MainPlaceHolder2_RVPageTemplateControl_panelMain"]/table/tbody/tr[3]/td[1]/a')
                    #sp1=select6.text.split("\n",1)[0]
                    if(sp==Winner.text):
                        
                        
                        List.append(Winner.text)
                        List.append('Batted first')

                    else:
                        
                        List.append(Winner.text)
                        List.append('Batted Second')
                else:
                    Winner=driver.find_element_by_xpath('//*[@id="ctl00_MainPlaceHolder2_RVPageTemplateControl_panelMain"]/table/tbody/tr[3]/td[3]/a')
                    if(sp==Winner.text):
                        
                        List.append(Winner.text)
                        List.append('Batted first')
                    else:
                        
                        List.append(Winner.text)
                        List.append('Batted Second')
            if table_index==0:  
                innings_played=driver.find_element_by_xpath('/html/body/form/div[3]/div[1]/div[1]/div[2]/div[1]/div[2]/div/div/div/div/div[2]/div[8]/div[2]/div[1]')
                List.append(innings_played.text)
            elif table_index==1:
                innings_played=driver.find_element_by_xpath('/html/body/form/div[3]/div[1]/div[1]/div[2]/div[1]/div[2]/div/div/div/div/div[2]/div[8]/div[3]/div[1]')
                List.append(innings_played.text)
            elif table_index==2:
                innings_played=driver.find_element_by_xpath('/html/body/form/div[3]/div[1]/div[1]/div[2]/div[1]/div[2]/div/div/div/div/div[2]/div[8]/div[6]/div[1]')
                List.append(innings_played.text)
            elif table_index==3:
                innings_played=driver.find_element_by_xpath('/html/body/form/div[3]/div[1]/div[1]/div[2]/div[1]/div[2]/div/div/div/div/div[2]/div[8]/div[7]/div[1]')
                List.append(innings_played.text)
                #time.sleep(0.5)
            # find total number of 4's

            ls=df[4].tolist() 
            ls.remove('4s')

            ls=[i for i in ls if i]
            T=0
            for i in range(0,len(ls)):
                T+=int(ls[i])
            List.append(T)
            
            #find totoal number of 6's
            TT=0
            l=df[5].tolist()
            l.remove('6s')

            l=[i for i in l if i]
            for i in range(0,len(l)):
                TT+=int(l[i])
            List.append(TT)
            


            #Overs
            overs=df[df[0].str.match('Overs')]
            overs=str(overs)
            overs=overs.split('Overs',1)[1]
            List.append(overs)
            
           #find extras
            tem=[]
            tem=df[df[0].str.match('Extras')] #find extras in data frame
            tem1=str(tem)
            #get last digit(s)afeter ... in data frame
            List.append(tem1.split('...',1)[1])
            #find score
        
            tem=(str(df[df[0].str.match('Total')]))
            tem1=[]
           
            tem2=tem.split('Total',1)#split on Total
           
            try:
                
                    tem2=str(tem2).split('/',)[1] #if 6/120 exists
                    tem2=str(tem2).split('\\n',1)[0] #split on \\n
                    tem2=str(tem2).split('\\',1)[0]#split on \
                    List.append(tem2)
            except:
                    List.append(tem2[1])#120
                
           
      
           # time.sleep(0.5)
     
  #maiden  overs
            data = []
            df=[]
            rows =  Bowling_table_body.find_all('tr')
            for row in rows:
                cols = row.find_all('td')
                cols = [ele.text.strip() for ele in cols]
                data.append([ele for ele in cols  ]) # Get rid of empty values
                df=pd.DataFrame(data)
            l=df[2].tolist()
            l.remove('M')
            TT=0
            l=[i for i in l if i]
            for i in range(0,len(l)):
                    TT+=int(l[i])
            
            List.append(TT)
  #---------------------------------------------------------------------------------# 
List=[]
Season = Select(driver.find_element_by_name('ctl00$MainPlaceHolder2$RVPageTemplateControl$rvsb$rvsbc_0$lc'))
Season.select_by_visible_text('2019/2020')




#print(rd)


for rnd in range(0,len(find_round(driver))): #for loop for Round-strart with 1 becoz 0 is Round text
  
    Season = Select(driver.find_element_by_name('ctl00$MainPlaceHolder2$RVPageTemplateControl$rvsb$rvsbc_0$lc'))
    Season.select_by_visible_text('2019/2020')
    
    Round = Select(driver.find_element_by_name('ctl00$MainPlaceHolder2$RVPageTemplateControl$rvsb$rvsbc_1$lc'))
    Round.select_by_index(rnd)



    
 

   
    for Gr in range(0,len(find_grade(driver))-1): #loop for Grade
        
        Season = Select(driver.find_element_by_name('ctl00$MainPlaceHolder2$RVPageTemplateControl$rvsb$rvsbc_0$lc'))
        Season.select_by_visible_text('2019/2020')

      
       # time.sleep(0.5)
        #select the Round
        Round = Select(driver.find_element_by_name('ctl00$MainPlaceHolder2$RVPageTemplateControl$rvsb$rvsbc_1$lc'))
        Round.select_by_index(rnd)

        Grade = Select(driver.find_element_by_name('ctl00$MainPlaceHolder2$RVPageTemplateControl$rvsb$rvsbc_3$lc'))
        Grade.select_by_index(Gr)


       
        driver.find_element_by_name('ctl00$MainPlaceHolder2$RVPageTemplateControl$rvsb$ctl09').click()

       
        try:
            for index in range(0,len(find_teams(driver))): #for loop for Teams
                  
                    #Match->Match URRL
                    List=[]


                    Season = Select(driver.find_element_by_name('ctl00$MainPlaceHolder2$RVPageTemplateControl$rvsb$rvsbc_0$lc'))
                    Season.select_by_visible_text('2019/2020')
                    List.append('2019/2020')


                    #time.sleep(0.5)                                                 
                    Round = Select(driver.find_element_by_xpath('//*[@id="ctl00_MainPlaceHolder2_RVPageTemplateControl_rvsb_rvsbc_3_lc"]'))
                    Round.select_by_index(rnd)


                    Grade = Select(driver.find_element_by_name('ctl00$MainPlaceHolder2$RVPageTemplateControl$rvsb$rvsbc_3$lc'))
                    Grade.select_by_index(Gr)

                    driver.find_element_by_name('ctl00$MainPlaceHolder2$RVPageTemplateControl$rvsb$ctl09').click()
                    rd=find_round(driver)

                    List.append(rd[rnd])
                    data=find_grade(driver)
                    List.append(data[Gr])
                    dt=find_teams(driver)
                    time.sleep(0.5)
                    dropdown = Select(driver.find_element_by_xpath('//*[@id="ctl00_MainPlaceHolder2_RVPageTemplateControl_rvsb_selectorBar_inner"]/div/div[2]/table/tbody/tr/td[2]/select'))
                    time.sleep(0.5)
                    List.append(dt[index])
                    dropdown.select_by_index(index)

                    #time.sleep(0.5)

                    batting_data(driver)
                    print("#####################################################################")
                    time.sleep(0.5)
                    print(List)
                    time.sleep(0.5)
                    # using list comprehension + zip() + slicing + enumerate() 
                    # Split list into lists by particular value 
                    size = len(List) 
                    idx_list = [idx + 1 for idx, val in
                    enumerate(List) ] 


                    res = [List[i: j] for i, j in
                    zip([0] + idx_list, idx_list + 
                    ([size] if idx_list[-1] != size else []))] 
              # print result 
                    #print("The list after splitting by a value : " + str(res)) 
                    #res=str(res)
                    #data_frame=pd.DataFrame(res,columns=['Season ','Grade','Round','Teams Played','Total 4S','Total 6s','Extras','Score','Overs','Total Maidens','Venue','Winning Team','Winner Batted','Total 4s','Total 6s','Extras','Score','Overs','Maaidens'])
                    time.sleep(1)
                    import csv
                    csvfile=open("E:\2018-19.csv",'w',newline='\n')
                    try:
                        out = csv.writer(csvfile)
                        out.writerow(map(lambda x: x, List))

                    finally:
                          csvfile.close()
                        #print(data_frame)
                    print('-----------------------------------------------')
                    time.sleep(1)

    #                 driver1 = webdriver.Firefox()
    #                 driver1.get(url)
    #                 driver=driver1

                    List=[]
        except:
                x=1



#####################################################################
['2019/2020', '  CURR', "  Kookaburra Women's Premier Firsts One Day", 'Essendon Maribyrnong Park v Box Hill']
-----------------------------------------------
#####################################################################
['2019/2020', '  CURR', "  Kookaburra Women's Premier Firsts One Day", 'Prahran v Dandenong', ' Sun 12 Jan 2020 ', 'Toorak Park', 'One day game', 'Prahran', 'Batted Second', '1ST INNINGS - DANDENONG', 1, 0, '  40.0        ', '  30        ', '  136        ', 6, ' Sun 12 Jan 2020 ', '1ST INNINGS - PRAHRAN', 8, 0, '  31.0        ', '  36        ', "137        ']", 2]
-----------------------------------------------
#####################################################################
['2019/2020', '  CURR', "  Kookaburra Women's Premier Firsts One Day", 'Ringwood v Carlton-Brunswick', ' Sun 12 Jan 2020 ', 'Russell Lucas Oval', 'One day game', 'Carlton-Brunswick', 'Batted Second', '1ST INNINGS - R